In [1]:
import pandas as pd
import media_mapper as mm
import numpy as np
from sklearn.cluster import KMeans

MAPBOX_API_KEY not found.
TWITTER_CONSUMER_KEY not found.
TWITTER_CONSUMER_SECRET not found.
TWITTER_ACCESS_TOKEN not found.
TWITTER_TOKEN_SECRET not found.
INSTAGRAM_CLIENT_ID not found.
INSTAGRAM_CLIENT_SECRET not found.
INSTAGRAM_ACCESS_TOKEN not found.
FLICKR_API_KEY not found.
FLICKR_API_SECRET not found.


In [2]:
###get dataframe
df = mm.pipeline.retrieve_sql_tweets('tweets_with_geoV6')

In [3]:
df.head()

,timestamp_ms,geoid10,id
0,1436990735491,060750615003006,621410300290592768
1,1437945824772,060750134001000,625416235082194944
2,1438041259509,060759804011003,625816517381332992
3,1437873474604,060750101001001,625112776483188736
4,1437929054749,060750604001007,625345896507666432


##get a count for every day of the week, and hour of the day

In [4]:
#obtain hour and day of week variables
df  =mm.pipeline.transform_timestamp(df, hour = True, DOW = True)

In [5]:
def tweet_rate_by_hour(df, dow = False):
    '''Calculates tweets per hour from a dataframe with unqiue entries for tweets, 
    and a datetime column. Drops the id column.
    If dow is false, will return a dataframe grouped only by hour. 
    If dow is true, will return a dataframe grouped by weekend(0) or weekday (1).'''
    #column of ones serves as a counter for number of tweets during group bys
    df['twt_cnt'] = 1
    #group by hour, so we have the total tweets for every hour
    df = df.groupby(['geoid10', 'date', 'hour']).agg(np.sum).reset_index().drop('id', 1)
    #At this point, the twt_count is the number of tweets for every hour 
    df['twt_rate'] = df['twt_cnt']
    if dow == False:
        #groupby geoid and hour to get the average rate for every hour
        df = df.groupby(['geoid10', 'hour']).agg(np.mean).reset_index()
        df.drop('DOW', 1, inplace = True)
        df.drop('twt_cnt', 1, inplace = True )  
        df['hr_bin'] = pd.cut(df.hour, bins = 5, labels = ['latenight', 'dawn','morning','afternoon','evening'])
        df = df.groupby(['geoid10', 'hr_bin']).agg(np.mean).reset_index().drop('hour', 1)
        return df
    if dow == True:
        #group by date for the average tweet rate every day
        df = df.groupby(['geoid10', 'date', 'DOW']).agg(np.mean).reset_index()
        df.drop('hour',1, inplace = True)
        df.drop('twt_cnt',1, inplace = True)    
        df['wknd']=df['DOW'].where(df['DOW'] > 5) 
        df['wkday'] = df['wknd'].isnull().astype(int)
        df.drop('DOW', 1, inplace = True)
        df.drop('wknd', 1, inplace = True)
        #final groupby to get an average count for week days or weekends 
        df = df.groupby(['geoid10', 'wkday']).agg(np.mean).reset_index()
        return df
    return df

In [6]:
def pivot_table(df, timevariable, column_prefix ):
    '''Takes a dataframe with three columns: Geoid, measurement of tweets, and time variable.
    Takes an argument of a column_prefix, which is used to name the new time variable columns.
    Pivots the table to create a dataframe with geoid as the index, and the time variable as the features.
    '''
    df = df.pivot(index = 'geoid10', columns = timevariable)
    df = df.reset_index(1)
    df = df.rename(columns = lambda x: column_prefix + str(x))
    df.columns = df.columns.droplevel()
    return df

In [7]:
#get tweet rate by hour
df_tr_wkdy = tweet_rate_by_hour(df, dow= True)
#pivot table so DOW is columns
df_tr_wkdy = pivot_table(df_tr_wkdy, 'wkday', column_prefix = 'wkd_')

In [8]:
df_tr_wkdy.tail()

wkday,wkd_,wkd_0,wkd_1
3790,060759809001128,1.0,1.083333
3791,060759809001129,NaN,1.000000
3792,060759809001134,1.0,1.000000
3793,060759901000002,1.0,NaN
3794,060759901000003,2.5,NaN


In [9]:
#get tweet rate by hour
df_tr_hr = tweet_rate_by_hour(df)
#pivot table so time of day is columns
df_tr_hr = pivot_table(df_tr_hr, 'hr_bin', column_prefix = 'hrbin_')

In [10]:
df_tr_hr.head()

hr_bin,hrbin_,hrbin_afternoon,hrbin_dawn,hrbin_evening,hrbin_latenight,hrbin_morning
0,060750101001000,1.566667,1.000000,1.497619,1.628571,1
1,060750101001001,2.148681,1.819597,3.529038,3.584762,1
2,060750101001002,NaN,NaN,NaN,1.000000,NaN
3,060750101001004,1.325000,1.232143,1.812762,2.202626,1
4,060750101001005,1.500000,1.000000,1.500000,1.000000,1


###merge dataframes for one matrix

In [11]:
dfk = pd.merge(df_tr_hr, df_tr_wkdy, left_on = 'hrbin_', right_on = 'wkd_').drop('wkd_', 1)

In [12]:
#fill missing values with zeroes 
dfk.fillna(0, inplace = True)

In [13]:
dfk.columns.tolist()

['hrbin_',
 'hrbin_afternoon',
 'hrbin_dawn',
 'hrbin_evening',
 'hrbin_latenight',
 'hrbin_morning',
 'wkd_0',
 'wkd_1']

In [14]:
k_array = dfk[['hrbin_afternoon','hrbin_dawn',
 'hrbin_evening',
 'hrbin_latenight',
 'hrbin_morning',
 'wkd_0',
 'wkd_1']].values

In [15]:
k_array.shape

(3795, 7)

In [16]:
kmeans = KMeans(n_clusters = 6)

In [17]:
kmeans.fit(X =k_array )

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=6, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [18]:
k_array.shape

(3795, 7)

In [19]:
features = dfk.columns.tolist()[1:]
geoids = dfk['hrbin_']

In [20]:
## 2. Print out the centroids.

In [21]:
print "cluster centers:"
print kmeans.cluster_centers_

cluster centers:
[[  0.4213016    0.18948981   0.48356415   0.44778998   0.16380134
    0.24426934   0.94300179]
 [ 83.58166667  86.95294118  84.99821429  88.72833333  84.97058824
   82.62385321  62.68055556]
 [  1.04345737   0.75056561   1.07498149   1.09614073   0.49188564
    1.60507619   1.06730977]
 [  1.71802006   1.88350404   2.02686733   1.95857963   0.93675727
    4.78074438   1.22297558]
 [ 43.15833333   1.52333333  45.11785714  11.39916667   2.28546178
   32.97701149   4.00548654]
 [ 15.40285714  25.4         32.70857143  30.12         1.41666667
   40.57142857   1.        ]]


# 3. Find the top 10 features for each cluster.

In [22]:
top_centroids = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]

In [23]:
print "top features for each cluster:"
for num, centroid in enumerate(top_centroids):
    print num
    print centroid
    print "%d: %s" % (num, ", ".join(features[i] for i in centroid))

top features for each cluster:
0
[6 2 3 0 5 1 4]
0: wkd_1, hrbin_evening, hrbin_latenight, hrbin_afternoon, wkd_0, hrbin_dawn, hrbin_morning
1
[3 1 2 4 0 5 6]
1: hrbin_latenight, hrbin_dawn, hrbin_evening, hrbin_morning, hrbin_afternoon, wkd_0, wkd_1
2
[5 3 2 6 0 1 4]
2: wkd_0, hrbin_latenight, hrbin_evening, wkd_1, hrbin_afternoon, hrbin_dawn, hrbin_morning
3
[5 2 3 1 0 6 4]
3: wkd_0, hrbin_evening, hrbin_latenight, hrbin_dawn, hrbin_afternoon, wkd_1, hrbin_morning
4
[2 0 5 3 6 4 1]
4: hrbin_evening, hrbin_afternoon, wkd_0, hrbin_latenight, wkd_1, hrbin_morning, hrbin_dawn
5
[5 2 3 1 0 4 6]
5: wkd_0, hrbin_evening, hrbin_latenight, hrbin_dawn, hrbin_afternoon, hrbin_morning, wkd_1


In [24]:
assigned_cluster = kmeans.transform(k_array).argmin(axis=1)

In [25]:
assigned_cluster

array([2, 3, 0, ..., 2, 0, 2])

###print out 30 random geoids for each cluster

In [49]:
from collections import defaultdict
geoidclust = defaultdict(int)
clusters = defaultdict(list)
for i in range(kmeans.n_clusters):
    cluster = np.arange(0, k_array.shape[0])[assigned_cluster==i]
    geoids = [dfk.ix[geoindx]['hrbin_'] for geoindx in cluster]
    print len(geoids), 'cluster #', i
    #make a dictionary of cluster as the key, and geoids as the list
    clusters[i] = geoids
    #second dictionary to quickly look up what cluster each geoid belongs to
    for geo in geoids:
        geoidclust[geo] = i
   

2094 cluster # 0
1 cluster # 1
1582 cluster # 2
116 cluster # 3
1 cluster # 4
1 cluster # 5


In [54]:
geoidclust['060750101001000']

2

###How do they Map?

In [44]:
dfk['geoid10'] = dfk['hrbin_']
dfk['tweetcnt'] = dfk[['hrbin_afternoon',
 'hrbin_dawn',
 'hrbin_evening',
 'hrbin_latenight',
 'hrbin_morning',
 'wkd_0',
 'wkd_1']].mean(axis = 1)

In [57]:
dfk.head()

,hrbin_,hrbin_afternoon,hrbin_dawn,hrbin_evening,hrbin_latenight,hrbin_morning,wkd_0,wkd_1,geoid10,twtcnt,tweetcnt
0,060750101001000,1.566667,1.000000,1.497619,1.628571,1,2.416667,1.111111,0750101001000,1.460091,1.460091
1,060750101001001,2.148681,1.819597,3.529038,3.584762,1,4.580645,1.840909,0750101001001,2.643376,2.643376
2,060750101001002,0.000000,0.000000,0.000000,1.000000,0,0.000000,1.000000,0750101001002,0.285714,0.285714
3,060750101001004,1.325000,1.232143,1.812762,2.202626,1,2.840000,1.445455,0750101001004,1.693998,1.693998
4,060750101001005,1.500000,1.000000,1.500000,1.000000,1,1.333333,1.000000,0750101001005,1.190476,1.190476


In [58]:
dfgeo = mm.pipeline.merge_shapes_with_dataframe(dfk)

In [ ]:
def add_properties_geo(row):
    '''Creates the properties geoid, tweetcnt, cluster number'''
    geoid = row['geoid10']
    tweetrate = row['tweetcnt']
    cluster =str(geoidclust[geoid])
    hexcolor = colors[cluster]
    geo_json = {"type": "Feature", "geometry": json.loads(row['geometry']),  "properties": {'geoid': geoid ,'tweetrate': tweetrate, 'cluster': cluster , 'hexcolor' : hexcolor}}
    return geo_json

def dataframe_to_geojson(df, outfilename):
    '''Takes in a dataframe with a count, geoid10, and list of tokens. Dumps it into a json geojason file'''
    df['geoid10'] = df['geoid10'].astype('str')
    df["tweetcnt"] = df['tweetcnt'].astype('str')
    list_to_export = []
    for idx, row in df.iterrows():
        list_to_export.append(add_properties_geo(row))
    with open(outfilename, 'w') as outfile:
        json.dump(list_to_export, outfile)

In [55]:
colors = ['#c6f50f', '#66cdaa', '#ffa500', '#f67453', '#f6c653' ,'#ab9eba']

In [60]:
dfgeo

,geometry,geoid10,hrbin_,hrbin_afternoon,hrbin_dawn,hrbin_evening,hrbin_latenight,hrbin_morning,wkd_0,wkd_1,twtcnt,tweetcnt
0,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",60750179021009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",60759806001043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",60759806001041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
3,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",60759806001038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
4,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",60750179021038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
5,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",60750179021023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
6,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",60750179021008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
7,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",60759806001039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
8,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",60759806001031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
9,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",60759806001032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
